In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import accuracy_score, precision_score
# Load the dataset
df = pd.read_csv('train.csv')
# Step 2: Splitting the Dataset into features
X = df.drop('label', axis=1) # Inputs (Features) , axis = 1 means to drop only one coloumn 
y = df['label']  # Output => (Target)
# Step 3: Splitting the dataset into 80% Training and 20% Validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to your data and transform it
X_train_normalized = scaler.fit_transform(X_train)
X_val_normalized = scaler.transform(X_val)
# Create DataFrames with the normalized data
df_train_normalized = pd.DataFrame(X_train_normalized, columns=X_train.columns)
df_val_normalized = pd.DataFrame(X_val_normalized, columns=X_val.columns)

# Define a custom PyTorch dataset class
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
# Create datasets and data loaders for training and validation
train_dataset = CustomDataset(df_train_normalized, y_train)
val_dataset = CustomDataset(df_val_normalized, y_val)

batch_size = 64  # You can adjust this based on your needs

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#creation of an archotecture
import torch.nn as nn

import torch.nn as nn

class DigitRecModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_rate=0.5):
        super(DigitRecModel, self).__init__()

        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.norm1 = nn.LayerNorm(hidden_size1)  # Add layer normalization
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.norm2 = nn.LayerNorm(hidden_size2)  # Add layer normalization
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        # Define the forward pass
        x = self.dropout1(self.relu1(self.norm1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.norm2(self.fc2(x))))
        x = self.fc3(x)
        return x



input_size = 28 * 28  # Assuming images are 28x28 pixels
output_size = 10  # Number of classes (digits 0-9)
dropout_rate = 0.5  
hidden_size1 = 256  
hidden_size2 = 128



simple_model = DigitRecModel(input_size,hidden_size1,hidden_size2, output_size, dropout_rate)
import torch
import torch.nn as nn
import torch.optim as optim

# Assume you have simple_model, train_loader, and val_loader defined

# losses and accuracy
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Experiment with different learning rates and batch sizes
learning_rates = [0.001, 0.01, 0.1, 0.5, 1.0]
batch_sizes = [32, 64, 128, 256, 512]
print("| Learning Rate | Batch Size | Epoch | Training Loss | Training Accuracy | Validation Loss | Validation Accuracy |")
for lr in learning_rates:
    for batch_size in batch_sizes:
        # Define model, loss function, and optimizer with the current learning rate
        simple_model = DigitRecModel(input_size,hidden_size1,hidden_size2, output_size, dropout_rate) # Replace YourModel with the actual model class
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(simple_model.parameters(), lr=lr)

        epochs = 3

        for epoch in range(epochs):
            simple_model.train()  # Set the model to training mode

            # Training
            correct_train = 0
            total_train = 0
            running_train_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = simple_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                running_train_loss += loss.item()

            train_accuracy = correct_train / total_train
            train_losses.append(running_train_loss / len(train_loader))
            train_accuracies.append(train_accuracy)

            # Validation
            simple_model.eval()  # Set the model to evaluation mode
            correct_val = 0
            total_val = 0
            running_val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    outputs = simple_model(inputs)
                    loss = criterion(outputs, labels)

                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).sum().item()
                    running_val_loss += loss.item()

            val_accuracy = correct_val / total_val
            val_losses.append(running_val_loss / len(val_loader))
            val_accuracies.append(val_accuracy)

            # Print the loss and accuracy after each epoch for the current learning rate and batch size
           

            print(f' {lr}\t\t    {batch_size}'
                  f'\t       {epoch + 1}/{epochs} '
                  f'\t {train_losses[-1]:.4f} \t    {train_accuracy:.4f} '
                  f'\t    {val_losses[-1]:.4f} \t      {val_accuracy:.4f}')


| Learning Rate | Batch Size | Epoch | Training Loss | Training Accuracy | Validation Loss | Validation Accuracy |
 0.001		    32	       1/3 	 2.0308 	    0.3012 	    1.3155 	      0.7412
 0.001		    32	       2/3 	 1.4850 	    0.5730 	    0.9155 	      0.8118
 0.001		    32	       3/3 	 1.1887 	    0.6769 	    0.7168 	      0.8427
 0.001		    64	       1/3 	 2.0371 	    0.2966 	    1.3564 	      0.7371
 0.001		    64	       2/3 	 1.5100 	    0.5725 	    0.9524 	      0.8054
 0.001		    64	       3/3 	 1.2196 	    0.6688 	    0.7463 	      0.8401
 0.001		    128	       1/3 	 2.0724 	    0.2865 	    1.3995 	      0.7410
 0.001		    128	       2/3 	 1.5365 	    0.5596 	    0.9656 	      0.8070
 0.001		    128	       3/3 	 1.2247 	    0.6649 	    0.7519 	      0.8342
 0.001		    256	       1/3 	 2.0175 	    0.3111 	    1.3762 	      0.7288
 0.001		    256	       2/3 	 1.5043 	    0.5674 	    0.9702 	      0.7994
 0.001		    256	       3/3 	 1.2050 	    0.6710 	    0.7556 	      0.8333
 0.